<b>MNIST digit recognition with LeNet</b>

In this practical session we will build a convolutional neural network that is able to recognise the digits 0-9 from an image.

You can run the code in a cell by selecting the cell and pressing Shift+Enter.

1) First, import some of the packages we will need:

Documentation for each of these packages can be found online: <br>
For numpy: https://docs.scipy.org/doc/numpy-dev/user/quickstart.html <br>
For matplotlib: http://matplotlib.org/api/pyplot_api.html <br>
For lasagne: http://lasagne.readthedocs.io <br>
For random: https://docs.python.org/2/library/random.html <br>

In [ ]:
import cPickle
import gzip
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import theano
import lasagne
import time
import random
random.seed(0)

2) Load the data. Download the data from: http://deeplearning.net/data/mnist/mnist.pkl.gz and save it somewhere on your disc, change the path in the second cell below to the location where you have saved it.

In [ ]:
def loadMNIST(path):
    f = gzip.open(path, 'rb')
    train_set, valid_set, test_set = cPickle.load(f)
    f.close()
    
    train_set_labels = train_set[1]
    train_set_images = np.resize(train_set[0],(len(train_set_labels),1,28,28))
    train_set_images = np.pad(train_set_images,((0,0),(0,0),(2,2),(2,2)),'constant', constant_values=0)
   
    valid_set_labels = valid_set[1]
    valid_set_images = np.resize(valid_set[0],(len(valid_set_labels),1,28,28))
    valid_set_images = np.pad(valid_set_images,((0,0),(0,0),(2,2),(2,2)),'constant', constant_values=0)

    test_set_labels = test_set[1]
    test_set_images = np.resize(test_set[0],(len(test_set_labels),1,28,28))
    test_set_images = np.pad(test_set_images,((0,0),(0,0),(2,2),(2,2)),'constant', constant_values=0)
    
    return train_set_labels, train_set_images, valid_set_labels, valid_set_images, test_set_labels, test_set_images

In [ ]:
train_set_labels, train_set_images, valid_set_labels, valid_set_images, test_set_labels, test_set_images = loadMNIST(r'D:\mnist\mnist.pkl.gz')

3) Now let's look at the data we've just loaded! How many samples are in each set? (Use .shape to see the dimensions) How large are the images? How many samples per class? Show some of the images with plt.imshow (use cmap='gray_r' for black digits on a white background and interpolation='none' to see the real pixels), you can access one of the training images as: train_set_images[i,0].

In [ ]:
#plt.imshow(train_set_images[0,0],cmap='gray_r',interpolation='none')

4) Convert the labels from a number between 0 and 9 to 'one-hot-encoding'. This means that for a label with number 3, there should be a 1 at element 3 and 0 everywhere else, i.e. [0, 0, 0, 1, 0, 0, 0, 0 ,0 ,0]. These are our target nodes, the node at position 3 should be active when the input image shows a 3. The code below does this for the training labels. 

Do the same for the validation labels!

In [ ]:
labels = np.zeros((len(train_set_labels),10),dtype=np.int16)        
for n in xrange(10):
    labels[:,n] = train_set_labels==n

print train_set_labels[:10]
print labels[:10]

#validlabels = np.zeros((len(valid_set_labels),10),dtype=np.int16)        
#for n in xrange(10):
#    validlabels[:,n] = valid_set_labels==n

5) Build the LeNet network. The print network.output_shape statement shows the dimensions after the current layer. Can you recognise all the elements of the network from the lecture?

In [ ]:
def buildLeNet(X1):
    network = lasagne.layers.InputLayer(shape=(None, 1, 32, 32),input_var=X1)    
    print network.output_shape     
    network = lasagne.layers.Conv2DLayer(network, num_filters=6, filter_size=(5,5), nonlinearity=lasagne.nonlinearities.rectify, W=lasagne.init.GlorotUniform())
    print network.output_shape 
    network = lasagne.layers.MaxPool2DLayer(network, pool_size=(2, 2))
    print network.output_shape 
    network = lasagne.layers.Conv2DLayer(network, num_filters=16, filter_size=(5,5), nonlinearity=lasagne.nonlinearities.rectify, W=lasagne.init.GlorotUniform())
    print network.output_shape 
    network = lasagne.layers.MaxPool2DLayer(network, pool_size=(2, 2))
    print network.output_shape 
    network = lasagne.layers.flatten(network)
    print network.output_shape 
    network = lasagne.layers.DenseLayer(network,num_units=120,nonlinearity=lasagne.nonlinearities.rectify)    
    print network.output_shape 
    network = lasagne.layers.DenseLayer(network,num_units=84,nonlinearity=lasagne.nonlinearities.rectify)
    print network.output_shape 
    network = lasagne.layers.DenseLayer(network,num_units=10,nonlinearity=lasagne.nonlinearities.softmax)     
    print network.output_shape 
    return network

In [ ]:
X = theano.tensor.tensor4()
Y = theano.tensor.matrix()
network = buildLeNet(X)

6) Define the functions for training the network. We will use categorical cross-entropy as loss function (second row) and stochastic gradient descent with momentum as optimiser (fourth row).

In [ ]:
outputtrain = lasagne.layers.get_output(network) #function that gets the output from the network defined before.
trainloss = lasagne.objectives.categorical_crossentropy(outputtrain, Y).mean() #function that computes the mean crossentropy between the output and the real labels.
params = lasagne.layers.get_all_params(network, trainable=True) #function that gets all the parameters (weights) in the network.
updates = lasagne.updates.momentum(trainloss, params, learning_rate=0.001) #function that performs an update of the weights based on the loss.
train = theano.function(inputs=[X, Y], outputs=trainloss, updates=updates, allow_input_downcast=True) #function that does all the above based on training samples X and real labels Y.

7) Define a function to validate the network.

In [ ]:
validate = theano.function(inputs=[X, Y], outputs=trainloss, allow_input_downcast=True) #function that computes the loss without performing an update

8) Define the functions for testing the network. deterministic=True turns off dropout in the network.

In [ ]:
outputtest = lasagne.layers.get_output(network, deterministic=True) #function that gets the output from the network defined before.
test = theano.function(inputs=[X], outputs=outputtest, allow_input_downcast=True) #function that gets the output based on input X

9) Do the training in random batches of a specific number of samples (we set the values below to 250 batches of 100 samples). You can use random.sample(a,n) to select n samples from array a. 

You can use the train(X,Y) function we have defined in 6) to perform an update of the network based on training images X and training labels Y, this function returns the loss. Save the loss of each training batch in the variable 'losslist' so we can look at them later (you can use .append()).

Also keep track of the loss for random batches from the validation set to see if your network is not overfitting (you can use validate(X,Y) from 7)).

<b>Remember</b> that if you restart the training process from the beginning you also need to reinitialise the network by running the cells starting from 5) again.

In [ ]:
trainingsamples = np.arange(len(train_set_labels)) #numbers from 0 until the number of samples
validsamples = np.arange(len(valid_set_labels))

minibatches = 250
minibatchsize = 100 

losslist = []
validlosslist = []

t0 = time.time()

for i in xrange(minibatches):
    #select random training en validation samples and perform training and validation steps here.
    
    #solution:
    #minibatchsamples = random.sample(trainingsamples,minibatchsize)                

    #loss = train(train_set_images[minibatchsamples],labels[minibatchsamples])
    #if (i+1)%50==0:
    #    print 'Loss minibatch {}: {}'.format(i,loss)
    #losslist.append(loss)            

    #validminibatchsamples = random.sample(validsamples,minibatchsize)           
    #validloss = validate(valid_set_images[validminibatchsamples],validlabels[validminibatchsamples])
    #if (i+1)%50==0:
    #    print 'Loss validation minibatch {}: {}'.format(i,validloss)
    #validlosslist.append(validloss)

t1 = time.time()
print 'Training time: {} seconds'.format(t1-t0)

9) Plot the loss curves for the training and validation sets (use plt.plot). Is 250 batches enough to train the network? How many do we need? What happens if you change the learning rate in 6)? What happens if you change the minibatchsize? What happens if you use another optimizer? Try to get the loss as low as possible! What happens if you make changes to the network? Use for example more or less filters or nodes, remove a layer, etc.

In [ ]:
#plt.figure()
#plt.plot(losslist)
#plt.plot(validlosslist)
#plt.legend(['Training loss','Validation loss'])

10) Evaluate the network on the test set with the test(X) function we have defined in 8). You can use np.argmax() to select the node with the highest probability. How well did it do? How many of the 10 000 test samples did it label correctly?

In [ ]:
#t0 = time.time()
#test_set_predictions = np.argmax(test(test_set_images),axis=1)
#t1 = time.time()
#print 'Testing time: {} seconds'.format(t1-t0)

In [ ]:
#TP = np.sum(test_set_labels==test_set_predictions)
#print 'Accuracy: {}'.format(float(TP)/float(len(test_set_labels)))

11) To see what is happening we can visualise the learned filters and their feature maps. 

First let's define the network again, but this time also return the result after the first layer.

In [ ]:
def buildLeNet(X1):
    network1 = lasagne.layers.InputLayer(shape=(None, 1, 32, 32),input_var=X1)  
    network1 = lasagne.layers.Conv2DLayer(network1, num_filters=6, filter_size=(5,5), nonlinearity=lasagne.nonlinearities.rectify, W=lasagne.init.GlorotUniform())
    network2 = lasagne.layers.MaxPool2DLayer(network1, pool_size=(2, 2))
    network2 = lasagne.layers.Conv2DLayer(network2, num_filters=16, filter_size=(5,5), nonlinearity=lasagne.nonlinearities.rectify, W=lasagne.init.GlorotUniform())
    network2 = lasagne.layers.MaxPool2DLayer(network2, pool_size=(2, 2))
    network2 = lasagne.layers.flatten(network2)
    network2 = lasagne.layers.DenseLayer(network2,num_units=120,nonlinearity=lasagne.nonlinearities.rectify)    
    network2 = lasagne.layers.DenseLayer(network2,num_units=84,nonlinearity=lasagne.nonlinearities.rectify)
    network2 = lasagne.layers.DenseLayer(network2,num_units=10,nonlinearity=lasagne.nonlinearities.softmax)      
    return network1, network2

In [ ]:
X = theano.tensor.tensor4()
Y = theano.tensor.matrix()
network1, network2 = buildLeNet(X)

In [ ]:
outputtrain = lasagne.layers.get_output(network2) 
trainloss = lasagne.objectives.categorical_crossentropy(outputtrain, Y).mean() 
params = lasagne.layers.get_all_params(network2, trainable=True) 
updates = lasagne.updates.momentum(trainloss, params, learning_rate=0.001) 
train = theano.function(inputs=[X, Y], outputs=trainloss, updates=updates, allow_input_downcast=True) 

validate = theano.function(inputs=[X, Y], outputs=trainloss, allow_input_downcast=True)

outputtest = lasagne.layers.get_output(network2, deterministic=True) 
test = theano.function(inputs=[X], outputs=outputtest, allow_input_downcast=True) 

We now define an additional function that obtains the feature maps after the first layer:

In [ ]:
outputlayer1 = lasagne.layers.get_output(network1) 
outputfeatures = theano.function(inputs=[X], outputs=outputlayer1, allow_input_downcast=True) 

Copy your training code below and retrain the network in the same way as before:

In [ ]:
#trainingsamples = np.arange(len(train_set_labels)) #numbers from 0 until the number of samples
#validsamples = np.arange(len(valid_set_labels))

#minibatches = 1000
#minibatchsize = 100 

#losslist = []
#validlosslist = []

#t0 = time.time()

#for i in xrange(minibatches):
    #select random training en validation samples and perform training and validation steps here.
    
    #solution:
    #minibatchsamples = random.sample(trainingsamples,minibatchsize)                
    
    #loss = train(train_set_images[minibatchsamples],labels[minibatchsamples])
    #if (i+1)%50==0:
    #    print 'Loss minibatch {}: {}'.format(i,loss)
    #losslist.append(loss)            

    #validminibatchsamples = random.sample(validsamples,minibatchsize)           
    #validloss = validate(valid_set_images[validminibatchsamples],validlabels[validminibatchsamples])
    #if (i+1)%50==0:
    #    print 'Loss validation minibatch {}: {}'.format(i,validloss)
    #validlosslist.append(validloss)

#t1 = time.time()
#print 'Training time: {} seconds'.format(t1-t0)

Now let's look at the feature maps after the first layer for one of the images from the test set. We have defined the function 'outputfeatures' for that:

In [ ]:
features = outputfeatures(test_set_images)

In [ ]:
print features.shape

In [ ]:
for i in xrange(6):
    plt.figure()
    plt.imshow(features[1,i],cmap='gray_r',interpolation='none')

Now let's look at the filters that are learned. We can use the lasagne function 'get_all_param_values' for that. These are the filters that are applied to the images to obtain the feature maps that we saw above.

In [ ]:
weights = lasagne.layers.get_all_param_values(network1)
filters = weights[0]
biases = weights[1]

In [ ]:
print filters.shape
print biases.shape

In [ ]:
for i in xrange(6):
    plt.figure()
    plt.imshow(filters[i,0],cmap='gray_r',interpolation='none')